# Hyperparameter Optimization

In [19]:
import optuna
import os
import joblib
import time
import datetime
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import precision_score, recall_score, f1_score
from dga.datasets import DomainDataset
from torch.utils.data import DataLoader
from dga.models.dga_classifier import DGAClassifier
import mlflow
import mlflow.pytorch
import optuna
import joblib

In [18]:
!pip install plotly

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11435 sha256=a657f81ea4e8bf8a9e83f563d27f632d981fe45ba93b999475652481f2e10755
  Stored in directory: c:\users\gilleal\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying


In [6]:
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    return xx_pad, yy, x_lens, y_lens


def prepare_dl(data_dir, filename, batch_size):
    df = pd.read_csv(os.path.join(data_dir, filename))
    ds = DomainDataset(df, train=True)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
    return dl


def seed_everything(seed, cuda=False):
    # Set the random seed manually for reproducibility.
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)


def calc_prec_rec(y_pred, y_actual):
    prec = precision_score(y_actual, y_pred)
    rec = recall_score(y_actual, y_pred)
    return prec, rec

In [7]:
def validate(model, epoch, valid_loader, criterion):
    model.eval()
    val_pred = np.array([])
    val_actual = np.array([])
    valid_loss = 0.
    with torch.no_grad():
        for batch_num, (x_padded, y_padded, x_lens, y_lens) in enumerate(valid_loader):
            output = model(x_padded, x_lens)
            val_pred = np.append(val_pred, output.detach().cpu().numpy())
            val_pred = np.round(val_pred).astype(int)
            val_actual = np.append(val_actual, y_padded).astype(int)
            loss = criterion(output, torch.Tensor(y_padded).unsqueeze(1))
            valid_loss += loss.item()

    accuracy = (val_pred == val_actual).sum() / len(valid_loader.dataset)
    prec, rec = calc_prec_rec(val_pred, val_actual)
    f1 = f1_score(val_actual, val_pred, average='micro')
    avg_loss = valid_loss / len(valid_loader)

    print(
        f"Validation | "
        f"Epoch {epoch:03} | "
        f"Loss {avg_loss:.3f} | "
        f"Accuracy {accuracy:.3f} | "
        f"Precision {prec:.3f} | "
        f"Recall {rec:.3f} | "
        f"F1 {f1:.3f} "
    )

    mlflow.log_metric(key="valid_accuracy", value=accuracy, step=epoch)
    mlflow.log_metric(key="valid_precision", value=prec, step=epoch)
    mlflow.log_metric(key="valid_recall", value=rec, step=epoch)
    mlflow.log_metric(key="valid_loss", value=valid_loss, step=epoch)
    mlflow.log_metric(key="valid_f1", value=f1, step=epoch)

    return f1


In [8]:
def train(log_interval, model, epoch, train_loader, optimizer, criterion):
    model.train()
    train_pred = np.array([])
    train_actual = np.array([])
    total_loss = 0.
    for batch_num, (x_padded, y_padded, x_lens, y_lens) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(x_padded, x_lens)
        loss = criterion(output, torch.Tensor(y_padded).unsqueeze(1))
        train_pred = np.append(train_pred, output.detach().cpu().numpy())
        train_pred = np.round(train_pred).astype(int)
        train_actual = np.append(train_actual, y_padded).astype(int)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if batch_num % log_interval == 0:
            print(f'Train Epoch: {epoch}' 
                    f'[{batch_num*len(x_padded)}/{len(train_loader.dataset)}'
                    f' ({100. * batch_num / len(train_loader):.0f}%)]'
                    f'\tLoss: {loss.item():.6f}')

    accuracy = (train_pred == train_actual).sum()/len(train_loader.dataset)
    prec, rec = calc_prec_rec(train_pred, train_actual)
    avg_loss = total_loss / len(train_loader)
    f1 = f1_score(train_actual, train_pred, average='micro')
    print(
        f"Train      | "
        f"Epoch {epoch:03} | "
        # f"Batch {batch_num:03} | "
        f"Loss {avg_loss:.3f} | "
        f"Accuracy {accuracy:.3f} | "
        f"Precision {prec:.3f} | "
        f"Recall {rec:.3f} | "
        f"F1 {f1:.3f} "
    )

    mlflow.log_metric(key="train_accuracy", value=accuracy, step=epoch)
    mlflow.log_metric(key="train_precision", value=prec, step=epoch)
    mlflow.log_metric(key="train_recall", value=rec, step=epoch)
    mlflow.log_metric(key="train_loss", value=avg_loss, step=epoch)


In [11]:
def hyper_train(trial):

    config = {
        'output_data_dir': '../data/',
        'model_dir': '../models/',
        'data_dir': '../data/processed/',
        'train_fn': 'mini_train.csv',
        'validation_fn': 'mini_validation.csv',
        'device': "cuda" if torch.cuda.is_available() else "cpu",
        'train_batch_size': 64,
        'test_batch_size': 1000,
        'epochs': 1,
        'seed': 0,
        'input_features': 68,
        'log_interval': 100,
        'save_model': False,
        'lr': trial.suggest_loguniform('lr', 1e-3, 1e-2),
        'dropout_rate': trial.suggest_uniform('dropout_rate', 0.1, 0.5),
        'hidden_dim': trial.suggest_int('hidden_dim', 10, 50),
        'embedding_dim': trial.suggest_int('embedding_dim', 3, 30),
        'n_layers': trial.suggest_int('n_layers', 2, 10),
        'optimizer': trial.suggest_categorical('optimizer', [optim.SGD, optim.RMSprop, optim.Adam]),
        'save_model': False
    }

    # Args holds all passed-in arguments
    device = torch.device(config['device'])
    cuda = torch.cuda.is_available()
    print("Using device {}.".format(device))
    torch.manual_seed(config['seed'])

    # Get loaders
    train_loader = prepare_dl(config['data_dir'], config['train_fn'], config['train_batch_size'])
    valid_loader = prepare_dl(config['data_dir'], config['validation_fn'], config['test_batch_size'])

    # Initialize DGA Classifier
    model = DGAClassifier(input_features=config['input_features'],
                          hidden_dim=config['hidden_dim'],
                          n_layers=config['n_layers'],
                          embedding_dim=config['embedding_dim'],
                          batch_size=config['train_batch_size'],
                          dropout_rate=config['dropout_rate'])

    if cuda:
        model.cuda()

    # Define loss and optimizer
    criterion = torch.nn.BCELoss()
    optimizer = config['optimizer'](model.parameters(), lr=config['lr'])

    train_start = time.time()  # start timer

    # Commence training
    with mlflow.start_run() as run:
        for key, value in config.items():
            mlflow.log_param(key, value)
        for epoch in range(1, config['epochs'] + 1):
            train(config['log_interval'], model, epoch, train_loader, optimizer, criterion)
            f1 = validate(model, epoch, valid_loader, criterion)

        # mlflow.pytorch.log_model(model, "models")

    train_finish = time.time()  # end timer
    duration_secs = train_finish - train_start
    duration_time = str(datetime.timedelta(seconds=duration_secs))
    print(f'[DURATION]: {duration_time}')

    # Save the model parameters
    if config["save_model"]:
        model_path = os.path.join(config['model_dir'], 'dga_model.pth')
        with open(model_path, 'wb') as f:
            torch.save(model.cpu().state_dict(), f)

    return f1

In [12]:
import datetime
now = datetime.datetime.now()
timestamp = str(now.strftime("%Y%m%d_%H-%M-%S"))
timestamp

'20201011_17-04-50'

In [13]:
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=hyper_train, n_trials=150)
now = datetime.datetime.now()
timestamp = str(now.strftime("%Y%m%d_%H-%M-%S"))
joblib.dump(study, f'../studies/{timestamp}_dga_optuna.pkl')

[I 2020-10-11 17:04:51,554] A new study created in memory with name: no-name-9a0ae3bc-d151-4997-8dd0-dba8d7005ba7


Using device cpu.
Train Epoch: 1[0/6185 (0%)]	Loss: 0.693663
Train      | Epoch 001 | Loss 0.555 | Accuracy 0.722 | Precision 0.745 | Recall 0.569 | F1 0.722 
Validation | Epoch 001 | Loss 0.467 | Accuracy 0.805 | Precision 0.835 | Recall 0.698 | F1 0.805 


[I 2020-10-11 17:05:06,350] Trial 0 finished with value: 0.8052325581395349 and parameters: {'lr': 0.0052114978077561335, 'dropout_rate': 0.33171384900036127, 'hidden_dim': 22, 'embedding_dim': 12, 'n_layers': 9, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 0 with value: 0.8052325581395349.


[DURATION]: 0:00:14.708218
Using device cpu.
Train Epoch: 1[0/6185 (0%)]	Loss: 0.722910
Train      | Epoch 001 | Loss 0.554 | Accuracy 0.729 | Precision 0.763 | Recall 0.562 | F1 0.729 
Validation | Epoch 001 | Loss 0.509 | Accuracy 0.733 | Precision 0.735 | Recall 0.620 | F1 0.733 


[I 2020-10-11 17:05:13,089] Trial 1 finished with value: 0.7325581395348836 and parameters: {'lr': 0.007998726256615954, 'dropout_rate': 0.4923939162740769, 'hidden_dim': 30, 'embedding_dim': 18, 'n_layers': 4, 'optimizer': <class 'torch.optim.rmsprop.RMSprop'>}. Best is trial 0 with value: 0.8052325581395349.


...

[I 2020-10-11 17:31:11,394] Trial 148 finished with value: 0.8444767441860465 and parameters: {'lr': 0.0012169434793672803, 'dropout_rate': 0.2963326602016402, 'hidden_dim': 31, 'embedding_dim': 27, 'n_layers': 5, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 96 with value: 0.9229651162790697.



[DURATION]: 0:00:10.334983
Using device cpu.
Train Epoch: 1[0/6185 (0%)]	Loss: 0.691983
Train      | Epoch 001 | Loss 0.431 | Accuracy 0.795 | Precision 0.815 | Recall 0.694 | F1 0.795 


[I 2020-10-11 17:31:20,897] Trial 149 finished with value: 0.9098837209302325 and parameters: {'lr': 0.001486012605667963, 'dropout_rate': 0.1946302605199313, 'hidden_dim': 38, 'embedding_dim': 24, 'n_layers': 5, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 96 with value: 0.9229651162790697.


Validation | Epoch 001 | Loss 0.239 | Accuracy 0.910 | Precision 0.912 | Recall 0.882 | F1 0.910 
[DURATION]: 0:00:09.454676


['../studies/20201011_17-31-20_dga_optuna.pkl']

In [14]:
from pathlib import Path

# Search the directory
list(Path('../studies').glob('*.*'))

[WindowsPath('../studies/20201011_17-31-20_dga_optuna.pkl'),
 WindowsPath('../studies/dga_optuna.pkl')]

In [15]:
study = joblib.load('../studies/20201011_17-31-20_dga_optuna.pkl')
df = study.trials_dataframe()
df.sort_values(by=['value'], ascending=False)

,number,value,datetime_start,datetime_complete,duration,params_dropout_rate,params_embedding_dim,params_hidden_dim,params_lr,params_n_layers,params_optimizer,state
96,96,0.922965,2020-10-11 17:20:47.575063,2020-10-11 17:20:56.051558,00:00:08.476495,0.211487,27,41,0.001245,5,<class 'torch.optim.adam.Adam'>,COMPLETE
137,137,0.915698,2020-10-11 17:28:26.635896,2020-10-11 17:28:38.410991,00:00:11.775095,0.168165,27,39,0.001794,5,<class 'torch.optim.adam.Adam'>,COMPLETE
83,83,0.915698,2020-10-11 17:18:32.599471,2020-10-11 17:18:42.173847,00:00:09.574376,0.313486,30,47,0.001549,5,<class 'torch.optim.adam.Adam'>,COMPLETE
125,125,0.915698,2020-10-11 17:26:10.104892,2020-10-11 17:26:20.185073,00:00:10.080181,0.185617,26,38,0.001192,5,<class 'torch.optim.adam.Adam'>,COMPLETE
101,101,0.911337,2020-10-11 17:21:37.551870,2020-10-11 17:21:47.211033,00:00:09.659163,0.188122,24,40,0.001288,5,<class 'torch.optim.adam.Adam'>,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
2,2,0.556686,2020-10-11 17:05:13.091640,2020-10-11 17:05:24.738541,00:00:11.646901,0.357542,27,42,0.001382,9,<class 'torch.optim.sgd.SGD'>,COMPLETE
46,46,0.556686,2020-10-11 17:11:58.636692,2020-10-11 17:12:06.675252,00:00:08.038560,0.423350,13,41,0.005442,6,<class 'torch.optim.sgd.SGD'>,COMPLETE
69,69,0.530523,2020-10-11 17:16:02.333080,2020-10-11 17:16:11.896839,00:00:09.563759,0.367371,28,41,0.001989,6,<class 'torch.optim.sgd.SGD'>,COMPLETE
20,20,0.500000,2020-10-11 17:08:09.548959,2020-10-11 17:08:17.774833,00:00:08.225874,0.203505,25,26,0.002582,3,<class 'torch.optim.sgd.SGD'>,COMPLETE


In [ ]:
# import plotly
# from optuna.visualization import plot_parallel_coordinate

# plot_parallel_coordinate(study)

In [ ]:
# from optuna.visualization import plot_contour

# plot_contour(study)